## llm_zoomcamp module 2 
#### 02-vector-search
import faq documents; run qdarnt vector search in docker; api call to openrouter.ai using key from .env file
model used for homework -- deepseek/deepseek-r1-0528:free

In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

docker run -p 6333:6333 -p 6334:6334    -v "$(pwd)/qdrant_storage:/qdrant/storage:z"    qdrant/qdrant

In [5]:
import sys
import os

# Get the parent directory of the current notebook
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
print(parent_dir)
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

print(sys.path)
from api_call import chat_with_openrouter

/home/ubuser/llm-zoomcamp
['/home/ubuser/llm-zoomcamp', '/home/ubuser/spark/spark-3.3.2-bin-hadoop3/python/lib/py4j-0.10.9.5-src.zip', '/home/ubuser/spark/spark-3.3.2-bin-hadoop3/python', '/home/ubuser/llm-zoomcamp/02-vector-search', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/ubuser/.local/lib/python3.10/site-packages', '/usr/local/lib/python3.10/dist-packages', '/usr/local/lib/python3.10/dist-packages/eflomal-0.1-py3.10-linux-x86_64.egg', '/usr/lib/python3/dist-packages', '/usr/lib/python3.10/dist-packages']


In [6]:
from qdrant_client import QdrantClient, models
qd_client = QdrantClient("http://localhost:6333")
EMBEDDING_DIMENSIONALITY = 512
model_handle = "jinaai/jina-embeddings-v2-small-en"
collection_name = "zoomcamp-faq"


In [7]:
qd_client.delete_collection(collection_name=collection_name)

False

In [8]:

qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

True

In [9]:
qd_client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword"
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [10]:
points = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [12]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=3, status=<UpdateStatus.COMPLETED: 'completed'>)

In [13]:
def vector_search(question):
    print('vector_search is used')
    
    course = 'data-engineering-zoomcamp'
    query_points = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=question,
            model=model_handle 
        ),
        query_filter=models.Filter( 
            must=[
                models.FieldCondition(
                    key="course",
                    match=models.MatchValue(value=course)
                )
            ]
        ),
        limit=5,
        with_payload=True
    )
    
    results = []
    
    for point in query_points.points:
        results.append(point.payload)
    
    return results

In [14]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [19]:
def rag(query, details=False):
    search_results = vector_search(query)
    prompt = build_prompt(query, search_results)
    answer = chat_with_openrouter(prompt)
    if details:
        return answer
    else:
        return answer['choices'][0]['message']['content']

In [16]:
query = "How do execute a command on a Kubernetes pod?"

In [20]:
rag('the course has already started, can I still enroll?')

vector_search is used


'Based solely on the provided FAQ context:\n\n**Yes, you can enroll and participate even after the course has started.**\n\nHere\'s why, according to the CONTEXT:\n\n1.  **Registration is not required for participation:** "You can also just start learning and submitting homework without registering. It is not checked against any registered list. Registration is just to gauge interest before the start date." (From *Course - I have registered for the Data Engineering Bootcamp...*)\n2.  **Specifically allows joining after start:** "Yes, even if you don\'t register, you\'re still eligible to submit the homeworks." (From *Course - Can I still join the course after the start date?*)\n3.  **Access to materials is maintained:** While focusing on post-course access, the answer for *Course - Can I follow the course after it finishes?* implies the materials and homework access remain open, supporting ongoing participation.\n\n**Important Note:** While enrollment and homework submission are open, 

# COHORT 2

#### Q1. Embedding the query

In [24]:
from fastembed import TextEmbedding, SparseTextEmbedding

model = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")
embeddings = list(model.embed(['I just discovered the course. Can I join now?']))

print(min(embeddings[0]))

-0.11726373885183883


In [25]:
import numpy as np
np.linalg.norm(embeddings)

1.0

In [27]:
v = embeddings[0]
v.dot(v)

1.0000000000000002

#### Q2. Cosine similarity with another vector

In [28]:
doc = 'Can I still join the course after the start date?'

In [30]:
emb = list(model.embed(doc))


In [31]:
v.dot(emb[0])

0.9008528895674548

#### Q3. Ranking by cosine

In [32]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [46]:
doc_texts = []
for doc in documents:
    doc_texts.append(doc.get('text'))


In [44]:
import numpy as np

doc_texts = [doc.get('text') for doc in documents]
doc_array = np.array(doc_texts).reshape(-1, 1)

In [48]:
emb_doc_texts = list(model.embed(doc_texts))

In [49]:
doc_array = np.array(emb_doc_texts).reshape(-1, 1)

In [52]:
for text in emb_doc_texts:
    print(text.dot(v))

0.7629684696540238
0.8182378150042889
0.8085397398734189
0.7133079015686243
0.7304499234333598
